# Application of amplitude estimation to Finances: Standard Approach

The reference for this notebook will be:

* NEASQC deliverable: *D5.1: Review of state-of-the-art for Pricing and Computation of VaR https://www.neasqc.eu/wp-content/uploads/2021/06/NEASQC_D5.1_Review-of-state-of-the-art-for-Pricing-and-Computation-of-VaR_R2.0_Final.pdf*

In finance one of the most important tasks is computing the "fair" price of a **derivative contract** whose defintion (following investopedia) is the following:

*A derivative is a contract between two or more parties whose value is based on an agreed-upon underlying financial asset (like a security) or set of assets (like an index).*

Usually, the problem of computing the price of a **derivative contract** can be reduced to computing an expectation of a given input function $f(x)$ when $x$ follows a proability density $p(x)$. 

$$\mathbb{E}[f]=\int_a^bp(x)f(x)dx$$

This integral can be approximated by the Riemann sum:

$$\mathbb{E}[f] = \sum_{i=0}^{2^n-1} p(x_i)f(x_i)dx$$

In this notebook we are going to show how to use the **Amplitude Estimation** algorithm to compute this values

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm
from copy import deepcopy

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from QQuantLib.utils.qlm_solver import get_qpu
QLMaaS = False
linalg_qpu = get_qpu(QLMaaS)

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results

## 1. Defining the problem

First thing we need to do for transforming our expectation computation in an **amplitude estimation** problem is defining the density probability $p(x)$ and the function to evaluate $f(x)$. We cannot work with a continuous variables so we need to discretize $p(x)$ and $f(x)$. 

We are going to define the following toy problem:

* Domain: our $x$ will be a set of $2^{n}$ integers numbers.

$$x \in \{0, 1, 2, ..., 2^n-1\}$$

* $p(x)$: Over our domain we are going to define a properly normalised density distribution in the form:

$$p(x)=\frac{x}{\sum_{i=0}^{2^{n}-1}i}$$

* $f(x)$: Over our domain we are going to define the following properly normalised function:

$$f(x) = \frac{x}{2^n-1}$$


**BE AWARE**

Is **MANDATORY** that $p(x)$ and $f(x)$ are properly normalised. Following conditions **must be** satisfied:

* For $p(x)$ is mandatory that: $\sum_{i=0}^{2^{n}} p(x_i) = 1$
* For $f(x)$ is mandatory that: $f(x_i) \leq 1 \forall i$

In [ ]:
#n will define the maximum numbers of our domain
n = 5
N = 2**n
#Domain
x = np.arange(N)
#p(x) density probability
p_X = x/np.sum(x)
#f(x) function to evaluate
f_X = x/np.max(x)

In [ ]:
#Testing normalised conditions!

print('p(x) condition: {}'.format(np.sum(p_X) == 1))
print('f(x) condition: {}'.format(np.max(f_X) <= 1))

In [ ]:
fig, ax1 = plt.subplots()
ax1.bar(x, p_X, label='p(x)')
ax1.set_ylabel('p(x)', color = 'b')
ax1.tick_params(axis='y', labelcolor='b')
ax2 = ax1.twinx()
ax2.plot(x, f_X, color='r', label = 'f(x)')
ax2.set_ylabel('f(x)', color='r')
ax2.tick_params(axis='y', labelcolor='r')
fig.legend(['p(x)', 'f(x)'])

## 2. Loading data

Second part is loading the data ($p(x)$ and $f(x)$) into the quantum state. Most widely approach used in the literature for this is creating following 2 loading operators:

1. $\mathcal{P}$. for loading $p(x)$
2. $\mathcal{F}$. for loading $f(x)$

So the main idea is begin with an initial $n+1$ qbits state:

$$|0\rangle \otimes|0\rangle_{n}$$

An apply the following loading operation:

$$|\Psi\rangle=\mathcal{F}\left(I\otimes\mathcal{P}\right)|0\rangle\otimes|0\rangle_{n}$$


We are going through this for our toy example:

* Probability loading:

$$\left(I\otimes\mathcal{P}\right)|0\rangle\otimes|0\rangle_{n} = |0\rangle\otimes \sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)}|i\rangle_{n}$$

* Function loading:

$$\mathcal{F}\left(I\otimes\mathcal{P}\right)|0\rangle\otimes|0\rangle_{n}= \mathcal{F} |0\rangle\otimes \sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)}|i\rangle_{n}=$$

$$ = |0\rangle\otimes \sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)f(x_i)}|i\rangle_{n} + |1\rangle\otimes\sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)(1-f(x_i))}|i\rangle_{n}$$

For doing this operations *load_probability* and *load_array* functions from **QQuantLib/DL/data_loading** module will be used (see notebook **01_DataLoading_Module_Use.ipynb**)

In [ ]:
from QQuantLib.DL.data_loading import load_probability, load_array

In [ ]:
p_gate = load_probability(p_X)
f_gate = load_array(np.sqrt(f_X))

In [ ]:
%qatdisplay p_gate --depth 0 --svg
%qatdisplay f_gate --depth 0 --svg

Now we compound the two operators $\mathcal{P}$ aqnd $\mathcal{F}$ for creating the necesary loading circuit

In [ ]:
toy_oracle = qlm.QRoutine()
toy_register = toy_oracle.new_wires(f_gate.arity)
toy_oracle.apply(p_gate, toy_register[:p_gate.arity])
toy_oracle.apply(f_gate, toy_register)

In [ ]:
%qatdisplay toy_oracle --depth 0 --svg

We can test if the loading was properly implemented. For doing this we are going to use **get_results** from *data_extracting* module

In [ ]:
toy_results,_,_,_,_ = get_results(toy_oracle, linalg_qpu=linalg_qpu)

In [ ]:
toy_results

We have loaded 
$$p(x)f(x) \forall x$$

into the following quantum states: 

$$|0\rangle\otimes \sqrt{p(x_i)f(x_i)}|i\rangle_{n}$$ 

where 

$$|i\rangle_{n} \in \{|0\rangle_{n}, |1\rangle_{n}, ..., |2^n-1\rangle_{n}\}$$. 

For testing the loading was done properly we need to test thhose **States** where the the first qbit is $|0\rangle$ (the $2^{n}$ first registers of the DataFrame)


In [ ]:
np.isclose(toy_results["Probability"][0:2**p_gate.arity], p_X*f_X).all()

Additionally the integral we are looking for is stored in: $|0\rangle\otimes \sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)f(x_i)}|i\rangle_{n}$. So we can use the **get_results** and only measure the last qbit for getting this integral:


In [ ]:
LastQbitState,_,_,_,_ = get_results(toy_oracle, linalg_qpu=linalg_qpu, qubits=[f_gate.arity-1])

In [ ]:
LastQbitState

As can be seen the integral is loaded in the amplitude of the $|0\rangle$ state of the last qbit

In [ ]:
np.isclose(LastQbitState['Probability'].iloc[0], sum(p_X*f_X))

## 3. Amplitude Estimation Problem.

When we finish the complete loading data our quantum state will be:

$$|\Psi\rangle = |0\rangle\otimes \sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)f(x_i)}|i\rangle_{n} + |1\rangle\otimes\sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)(1-f(x_i))}|i\rangle_{n}$$

Now we can define two orthogonal states  $|\Psi_{1}\rangle$ y $|\Psi_{0}\rangle$ in the following way:

$$\sqrt{a}|\Psi_{0}\rangle = |0\rangle \otimes\sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)f(x_i)}|i\rangle_{n}$$

and

$$\sqrt{1-a}|\Psi_{1}\rangle = |1\rangle\otimes\sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)(1-f(x_i))}|i\rangle_{n}$$


So our $|\Psi\rangle$ will be:

$$|\Psi\rangle=\sqrt{a}|\Psi_{0}\rangle+\sqrt{1-a}|\Psi_{1}\rangle$$

The probability of measuring $|0\rangle$ in the leftmost qubit is:

$$ a = \sum_{i=0}^{2^{n}-1}\left|\sqrt{p(x_i)f(x_i)}\right|^2$$

So we have transformed the expectation value computation in an **amplitude estimation** problem and we can use all the clasess we have programed for solving it. We are going to solve this toy **amplitude estimation** problem using the following modules:

1. Maximum Likelihood Amplitude Estimation (MLAE)
2. Amplitude Estimation using classical Quantum Phase Estimation (CQPEAE)
3. Amplitude Estimation using iterative Quantum Phase Estimation (IQPEAE)
4. Iterative Quantum Amplitude Estimation (IQAE)

For using the different classes for solving our toy problem we need to provide always:

* Oracle (*toy_oracle*)
* target: in the toy case this will be the $|0\rangle$ state of the last qbit
* index: index of the last qbit

In [ ]:
target = [0]
index = [toy_oracle.arity-1]

### 3.1 MLAE

In [ ]:
from QQuantLib.AE.maximum_likelihood_ae import MLAE

In [ ]:
mlae_dict = {
    'qpu': linalg_qpu,
}
mlae = MLAE(
    toy_oracle,
    target = target,
    index = index, 
    **mlae_dict
)

mlae_a = mlae.run()
print('mlae_a: ', mlae_a)

In [ ]:
mlae.circuit_statistics

In [ ]:
mlae.run_time

In [ ]:
mlae.time_pdf

### 3.2 cQPE_AE

In [ ]:
from QQuantLib.AE.ae_classical_qpe import CQPEAE

In [ ]:
ae_cqpe_dict = {
    'qpu': linalg_qpu,
    'auxiliar_qbits_number': 8,
    'shots': 100
}

ae_cqpe = CQPEAE(
    toy_oracle,
    target = target,
    index = index, 
    **ae_cqpe_dict
)
ae_cqpe_a  = ae_cqpe.run()

print('ae_cqpe_a: ', ae_cqpe_a)

In [ ]:
ae_cqpe.circuit_statistics

In [ ]:
ae_cqpe.cqpe.time_pdf

In [ ]:
ae_cqpe.run_time

### 3.3 IQPE_AE

In [ ]:
from QQuantLib.AE.ae_iterative_quantum_pe import IQPEAE

In [ ]:
ae_iqpe_dict = {
    'qpu': linalg_qpu,
    'cbits_number': 8,
    'shots': 10
}

ae_iqpe = IQPEAE(
    toy_oracle,
    target = target,
    index = index, 
    **ae_iqpe_dict
)

ae_iqpe_a  = ae_iqpe.run()

print('ae_iqpe_a: ', ae_iqpe_a)

In [ ]:
ae_iqpe.circuit_statistics

In [ ]:
ae_iqpe.iqpe_object.time_pdf

In [ ]:
ae_iqpe.run_time

### 3.4 IQAE

In [ ]:
from QQuantLib.AE.iterative_quantum_ae import IQAE

In [ ]:
iqae_dict = {
    'qpu': linalg_qpu,
    #'shots': 10
}

iqae = IQAE(
    toy_oracle,
    target = target,
    index = index, 
    **iqae_dict
)

iqae_a = iqae.run()

print('iqae_a: ', iqae_a)


In [ ]:
iqae.circuit_statistics

In [ ]:
iqae.time_pdf

In [ ]:
iqae.run_time

## SUMMARY

In [ ]:
methods = ['MLAE', 'CQPEAE', 'IQPEAE', 'IQAE']
a_estimated = [mlae.ae, ae_cqpe.ae, ae_iqpe.ae, iqae.ae]
Expectation_Integral = sum(p_X*f_X)
a_error = [
    abs(mlae.ae-Expectation_Integral), 
    abs(ae_cqpe.ae-Expectation_Integral), 
    abs(ae_iqpe.ae-Expectation_Integral),
    abs(iqae.ae-Expectation_Integral),
]


dic_staff = {
    'Estimated_a': a_estimated,
    'Expectation': Expectation_Integral,
    'Error_a': a_error
}

In [ ]:
Results = pd.DataFrame(dic_staff, index=methods)

In [ ]:
Results